# Кластеризация существительных по левому глагольному контексту
Наша гипотеза состоит в том, что мы можем с помощью алгоритма кластеризации найти группы глаголов, которые ведут себя сходным образом. Для ее проверки мы исследуем существительные в подходящих контекстах: биграммы с переходными глаголами и существительными в соответствующих падежах (кроме именительного и предложного) и триграммы с переходными глаголами, предлогом и существительным.

## Данные
Данные взяты из НКРЯ (поиск по биграммам и триграммам).
### Запрос:
Обращение к данным происходит путем подстановки леммы в заранее сформированный запрос (см. reference.txt), извлечение данных - с помощью XPath.

### Существительные:
В качестве леммы в запросе к корпусу использовались первые 500 существительных из Частотного словаря русского языка, а также те слова, которые были получены в результате прошлого интерактива (если быть точным - их объединение, порядка 900 слов).

### Выход:
Мы получаем данные в виде html-страницы с таблицей, строки которой имеют вид:

In [5]:
import lxml.html
import urllib.request
import urllib.error
import urllib.parse
import pymorphy2
import json

morph = pymorphy2.MorphAnalyzer()

BIGRAM_LINK = "http://search.ruscorpora.ru/search.xml?env=sas1_2&mycorp=&mysent=&mysize=&mysentsize=" \
              "&dpp=100&spp=100&spd=100&text=lexgramm&mode=ngrams_2_lexgr&sort=gr_freq&lang=ru&nodia=1" \
              "&parent1=0&level1=0&lex1=&gramm1=V%2Ctran&flags1=&parent2=0&level2=0&min2=1&max2=1&lex2={0}" \
              "&gramm2=%28gen%7Cgen2%7Cdat%7Cacc%7Cacc2%7Cins%29&flags2="
TRIGRAM_LINK = "http://search.ruscorpora.ru/search.xml?env=sas1_2&mycorp=&mysent=&mysize=&mysentsize=" \
               "&dpp=100&spp=100&spd=100&text=lexgramm&mode=ngrams_3_lexgr&sort=gr_freq&lang=ru&nodia=1" \
               "&parent1=0&level1=0&lex1=&gramm1=V&flags1=&parent2=0&level2=0&min2=1&max2=1&lex2=&gramm2=" \
               "PR&flags2=&parent3=0&level3=0&min3=1&max3=1&lex3={0}&gramm3=&flags3="

agent_name = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
            
            
# загрузка страниц
def load_page(url, encoding="cp1251"):
    """
    Простая функция для загрузки страницы с сайта

    :param url: URI of the page
    :param encoding: page encoding
    :return: tuple: a boolean showing success, content of the page (or error message), and http code if available (or 0)
    """
    try:
        req = urllib.request.Request(urllib.parse.quote(url, safe=":/&=?%"), headers={'User-Agent': agent_name})
        with urllib.request.urlopen(req) as r:
            code = r.getcode()
            page = r.read().decode(encoding)
            loaded = True
    except urllib.error.HTTPError as e:
        page = e.reason
        code = e.code
        loaded = False
    except urllib.error.URLError as e:
        page = e.reason
        code = 0
        loaded = False
    except Exception as e:
        page = str(e)
        code = 0
        loaded = False
    return loaded, page, code

def load_words():
    """
    Загрузка списка слов
    """
    with open("w1.txt", "r") as f:
        txt = f.read()
        words1 = txt.split("\n")
        words1 = [w.strip().lower() for w in words1]
    with open("w2.txt", "r") as f:
        txt = f.read()
        words2 = txt.split("\n")
        words2 = [w.strip().lower() for w in words2]
    words = set(words1 + words2)
    return words

def create_link(word, is_trigram=False):
    """
    Создает ссылки на страницы в соответствии с необходимым запросом
    """
    if is_trigram:
        return TRIGRAM_LINK.format(word)
    else:
        return BIGRAM_LINK.format(word)

def lemma(token, morph):
    """
    Получаем начальную форму и заодно ищем наиболее вероятный глагольный разбор токена.
    Если глагольного разбора нет - возвращаем False.
    """
    parse = morph.parse(token)
    for p in parse:
        if 'VERB' in p.tag:
            return True, p.normal_form
    return False, ""

def extract_words_freqs(bi_page, tri_page):
    """
    Извлекает из текста страницы глаголы/глаголы с предлогами и возвращает в виде словаря, где глаголы - ключи, частоты - 
    значения.
    """
    output_dict = dict()
    bi_tree = lxml.html.fromstring(bi_page)
    # обходим таблицу на странице по строкам
    for tr in bi_tree.iter('tr'):
        # прямо в ячейках живут цифры (номер пп. и частотность)
        td_text = tr.xpath(".//td/text()")
        if not td_text:
            continue
        # в тегах span живут слова
        span_text = tr.xpath(".//td/span/text()")
        freq = int(td_text[1])
        token = span_text[0]
        is_verb, word = lemma(token, morph)
        if is_verb:
            if word in list(output_dict.keys()):
                output_dict[word] = output_dict[word] + freq
            else:
                output_dict[word] = freq
    tri_tree = lxml.html.fromstring(tri_page)
    # переходим к триграммам
    for tr in tri_tree.iter('tr'):
        td_text = tr.xpath(".//td/text()")
        if not td_text:
            continue
        # в тегах span живут слова
        span_text = tr.xpath(".//td/span/text()")
        freq = int(td_text[1])
        token = span_text[0]
        is_verb, word = lemma(token, morph)
        prep_phrase = word + " " + span_text[1]
        if is_verb:
            if prep_phrase in list(output_dict.keys()):
                output_dict[prep_phrase] = output_dict[prep_phrase] + freq
            else:
                output_dict[prep_phrase] = freq
    return output_dict

def loader(words):
    """
    Составляем конечный словарь
    """
    output_dict = dict()
    counter = 1
    for word in words:
        print("Dumping data for {0}.".format(word))
        bi_loaded, bi_page, code = load_page(create_link(word, False))
        tri_loaded, tri_page, code = load_page(create_link(word, True))
        if bi_loaded and tri_loaded:
            output_dict[word] = extract_words_freqs(bi_page, tri_page)
            print("[{0}] Completed.".format(str(counter)))
            counter += 1
    return output_dict

def dumper():
    """
    Запускаем процесс загрузки биграммов-триграммов и сохраняем результаты в json
    """
    print("Starting the job")
    words = list(load_words())
    print("Wordlist is loaded: {0} entries".format(str(len(words))))
    loaded = loader(words)
    with open("out.json", "w") as w:
        json.dump(loaded, w)
    return loaded
    
    

### Результат
Мы имеем файл out.json, в котором сериализован словарь с данными. Данные представлены в виде словаря, ключами являются существительные, а значениями - еще один словарь: ключи - глаголы или биграммы глагол + предлог, а значения - количество их вхождений в корпус. 
### Предобработка данных
Теперь необходимо преобразовать эти данные в более удобный для обработки вид. Представим каждое существительное как вектор в пространстве глагольных униграмм и биграмм, значениями каждой координаты будет количество вхождений в корпус.

In [1]:
import pandas as pd
import json

# Загружаем json
with open("out.json", "r") as f:
    verb_dict = json.load(f)

del verb_dict[''] # ошибочное значение в датасете
    
# Преобразовываем словарь в датафрейм
verb_data = pd.DataFrame.from_dict(verb_dict, orient="index", dtype=int)

# Избавляемся от na-значений
verb_data = verb_data.fillna(0)
verb_data.describe()

,думать с,сказать с,засмеяться от,причмокнуть от,изъявить,работать с,рычать от,выражать,улыбаться от,чувствовать,...,служить за,вовлекать,возникнуть у,прибыть между,вербовать,ознакомить,чижать,поглядеть-ка,приготовить за,посудить
count,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,...,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000,852.000000
mean,0.005869,0.116197,0.059859,0.004695,0.390845,0.177230,0.003521,0.475352,0.009390,0.517606,...,0.004695,0.001174,0.008216,0.004695,0.001174,0.005869,0.002347,0.001174,0.002347,0.001174
std,0.171297,1.296971,1.146162,0.137038,11.034336,2.977834,0.102778,5.558204,0.199662,5.927959,...,0.137038,0.034259,0.239816,0.137038,0.034259,0.171297,0.068519,0.034259,0.068519,0.034259
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,21.000000,24.000000,4.000000,322.000000,84.000000,3.000000,118.000000,5.000000,145.000000,...,4.000000,1.000000,7.000000,4.000000,1.000000,5.000000,2.000000,1.000000,2.000000,1.000000


## Кластеризация
Для кластеризации мы воспользуемся алгоритмами KMeans, AgglomerativeClustering и DBSCAN. Мы запустим их на полном датасете и на данных уменьшенной размерности, полученных методом главных компонент. Количество кластеров установим в 20. 

In [2]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# стандартизируем столбцы
std = StandardScaler()
verb_data_std = std.fit_transform(verb_data)

# метод главных компонент
pca = PCA(10)
verb_reduced = pca.fit_transform(verb_data)
print(pca.explained_variance_ratio_)

[ 0.26308901  0.22304113  0.09831955  0.06503624  0.06160445  0.04503541
  0.02006368  0.0185175   0.01703503  0.01584853]


In [119]:
clt = KMeans(50)
clusters_K = clt.fit_predict(verb_reduced)
clt = AgglomerativeClustering(50)
clusters_A = clt.fit_predict(verb_reduced)

In [121]:
print("KMeans clusters")
for i in range(50):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(100):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))


KMeans clusters
Cluster 0
абонемент аквариум актер акция аллергия аперитив аппетит аптека бабочка бабушка база бакалея балахон балкон бампер банкет бахилы безопасность белка белье берег берет бинт блюдо бмв бой бокал болото боль борщ борьба бра бриджи будильник будка бургер бутерброд бюджет ванна варежки велосипед веник вентилятор весы ветер вешалка взгляд вилка вирус витрина внук водитель воздух возраст войско волос воротник враг врач выборы выписка высота выходной газель галстук гардина гейзер гипс глубина говядина гололед горячее готовка градусник гражданин график гроза гром грудь грязь губа деверь дед дедлайн декор депутат десерт детство джем джинсовка диагноз диван директор доктор долина доля домофон дрессировка дядя егерь жаба жара жарить жарка жилет журнал заливное застолье защита звук здание здоровье зона зрение зять игра игрушка игуана иней институт интубация инъекция использование исследование итог каблук канарейка канатка капельница капитан карман картошечка касса кассир кас

Окей. Результат не то чтобы умопомрачительный, но кое-какие кластеры все-таки были выделены: например, в один кластер попали члены семьи: брат, дочь, сын, сестра - а в другой вполне абстрактные понятия: влияние, власть, представление, время. DBSCAN справился гораздо хуже. Попробуем запустить на полном датасете.

In [90]:
clt = KMeans(20)
clusters_K = clt.fit_predict(verb_data)
clt = AgglomerativeClustering(20)
clusters_A = clt.fit_predict(verb_data)

In [91]:
print("KMeans clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))

KMeans clusters
Cluster 0
абонемент автомобиль автор аквариум акт актер акция аллергия анализ аперитив аппетит аптека армия бабочка бабушка база бакалея балахон балкон бампер банк банкет бахилы безопасность белка белье берег берет бизнес билет бинт блюдо бмв бог бой бокал болезнь болото боль большинство борщ борьба бра брат бриджи будильник будка будущее бумага бургер бутерброд бутылка бюджет ванна варежки вариант век велосипед веник вентилятор вера весы ветер вечер вешалка вещь взгляд вилка вирус витрина вкус власть влияние внук вода водитель воздух возраст война войско волос воля вопрос воротник впечатление враг врач встреча второе выбор выборы вывод выписка выражение высота выход выходной газ газель газета галстук гардина гейзер генерал герой гипс глава глубина говядина гололед голос гонка гора город горячее господин гость государство готовка градусник гражданин граница график гроза гром грудь группа грязь губа данные дверь движение двор деверь девочка девушка дед дедлайн действие д

Снова неудача. Последняя попытка - попробуем обучиться на стандартизированных значениях:

In [92]:
clt = KMeans(20)
clusters_K = clt.fit_predict(verb_data_std)
clt = AgglomerativeClustering(20)
clusters_A = clt.fit_predict(verb_data_std)

In [94]:
print("KMeans clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))

KMeans clusters
Cluster 0
население
Cluster 1
офицер
Cluster 2
представление
Cluster 3
абонемент автомобиль автор аквариум акт актер акция аллергия анализ аперитив аппетит аптека армия бабочка бабушка база бакалея балахон балкон бампер банк банкет бахилы безопасность белка белье берег берет бизнес билет бинт блюдо бмв бог бой бокал болезнь болото боль большинство борщ борьба бра брат бриджи будильник будка будущее бумага бургер бутерброд бутылка бюджет ванна варежки вариант век велосипед веник вентилятор вера весы ветер вечер вешалка вещь взгляд вид вилка вирус витрина вкус власть влияние внимание внук вода водитель воздух возможность возраст война войско воля вопрос воротник впечатление враг врач время встреча второе выбор выборы вывод выписка выражение высота выход выходной газ газель газета галстук гардина гейзер генерал герой гипс глава глаз глубина говядина год голова гололед голос гонка гора город горячее господин гость государство готовка градусник гражданин граница график гроза

Собственно, содержимое последнего кластера очень неплохо описывает то, что мы сейчас испытываем. Как насчет того, чтобы заняться feature engineering? Попробуем уменьшить количество признаков, оставив лишь те, значение которых больше n.

In [104]:
def reparse(fdict, n):
    """
    Удаляем все вхождения во вложенном словаре, значение которого меньше n
    """
    for word in list(fdict.keys()):
        for feature in list(fdict[word].keys()):
            if fdict[word][feature] < n:
                del fdict[word][feature]
                
reparse(verb_dict, 15)

In [106]:
# Преобразовываем словарь в датафрейм
verb_data = pd.DataFrame.from_dict(verb_dict, orient="index", dtype=int)

# Избавляемся от na-значений
verb_data = verb_data.fillna(0)

# стандартизируем столбцы
verb_data_std = std.fit_transform(verb_data)

# метод главных компонент
verb_reduced = pca.fit_transform(verb_data)
print(pca.explained_variance_ratio_)
verb_data.describe()

[ 0.26313782  0.22295395  0.09866479  0.06524688  0.0617956   0.04516769
  0.01996631  0.01857629  0.0170742   0.01589448]


,влезть на,стоять на,вскочить на,сидеть на,сесть на,усесться на,лежать на,спрыгнуть с,присесть на,взять с,...,ездить за,приехать из-за,возвратиться из-за,пересекать,оказаться за,пережить,устать от,погибнуть на,убить на,приготовить
count,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,...,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000
mean,0.054348,3.518634,1.212733,3.481366,2.777950,0.192547,4.574534,0.068323,0.434783,0.037267,...,0.110248,0.041925,0.032609,0.045031,0.045031,0.023292,0.029503,0.094720,0.035714,0.043478
std,0.978063,25.547589,28.330472,27.828199,26.208022,3.301850,35.670634,1.245205,6.108437,0.945732,...,2.797792,1.063949,0.827516,1.142760,1.142760,0.591083,0.748705,2.403737,0.906327,1.103355
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19.000000,404.000000,718.000000,405.000000,435.000000,77.000000,425.000000,26.000000,140.000000,24.000000,...,71.000000,27.000000,21.000000,29.000000,29.000000,15.000000,19.000000,61.000000,23.000000,28.000000


In [107]:
# Уменьшили количество фичей в 3 раза

clt = KMeans(20)
clusters_K = clt.fit_predict(verb_reduced)
clt = AgglomerativeClustering(20)
clusters_A = clt.fit_predict(verb_reduced)

In [108]:
print("KMeans clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))

KMeans clusters
Cluster 0
автомобиль автор акт актер акция аллергия анализ аппетит аптека армия бабочка бабушка база балкон банк банкет безопасность белье берег берет бизнес билет блюдо бог бой бокал болезнь болото боль большинство борщ борьба будка будущее бумага бутерброд ванна велосипед вера ветер вечер вешалка взгляд вилка вирус витрина вкус внук вода водитель воздух возраст война войско волос воротник впечатление враг врач встреча второе выбор выборы вывод выписка выражение высота выход выходной газ газета галстук генерал герой глава глаз глубина голос гонка гора город горячее господин гость государство граница гроза гром грудь грязь губа данные дверь движение двор девочка девушка дед действие деревня дерево детство деятельность диагноз диван директор договор доктор документ должность долина доллар доля дорога доход друг дух душа дядя жара желание жена журнал завод задача закон зал зарплата защита заявление звезда звук здание здоровье земля зима знак знание зона зрение зуб игра ид

In [115]:
# Уменьшили количество фичей в 3 раза

clt = KMeans(50)
clusters_K = clt.fit_predict(verb_data)
clt = AgglomerativeClustering(100)
clusters_A = clt.fit_predict(verb_data)

print("KMeans clusters")
for i in range(50):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(100):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))

KMeans clusters
Cluster 0
власть влияние воля деньги знак имя ответ повод покой представление слово совет счастье удовольствие форма характер
Cluster 1
день
Cluster 2
право
Cluster 3
вид
Cluster 4
голова
Cluster 5
возможность
Cluster 6
внимание
Cluster 7
месяц раз
Cluster 8
значение смысл
Cluster 9
плечо
Cluster 10
рука
Cluster 11
участие
Cluster 12
основание отношение понятие сила случай успех цель
Cluster 13
чай
Cluster 14
вопрос
Cluster 15
автомобиль автор актер акция аллергия анализ аппетит аптека армия бабочка бабушка база балкон банк банкет безопасность белье берег берет бизнес билет блюдо бог бой бокал болезнь болото боль большинство борщ борьба брат будка будущее бумага бутерброд бутылка ванна вариант велосипед вера ветер вечер вешалка взгляд вилка вирус витрина вкус внук вода водитель воздух возраст войско волос воротник враг врач встреча второе выборы выписка высота выходной газ галстук генерал герой глава глубина гонка гора горячее господин гость государство граница гроза гр

In [112]:
# Наконец, попробуем уменьшить количество фичей с помощью иерархической классификации
from sklearn.cluster import FeatureAgglomeration
fa = FeatureAgglomeration(20)
agglomerated_data = fa.fit_transform(verb_data)

clt = KMeans(20)
clusters_K = clt.fit_predict(agglomerated_data)
clt = AgglomerativeClustering(20)
clusters_A = clt.fit_predict(agglomerated_data)

print("KMeans clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_K==i]))
print()
print("Agglomerative clusters")
for i in range(20):
    print("Cluster " + str(i))
    print(" ".join(verb_data.index[clusters_A==i]))

KMeans clusters
Cluster 0
автомобиль автор акт актер акция аллергия анализ аппетит аптека армия бабочка бабушка база балкон банк банкет безопасность белье берег берет бизнес билет блюдо бог бой бокал болезнь болото боль большинство борщ борьба брат будка будущее бумага бутерброд бутылка ванна вариант век велосипед вера ветер вечер вешалка взгляд вилка вирус витрина вкус влияние внук вода водитель воздух возраст война войско волос воля воротник впечатление враг врач встреча второе выбор выборы вывод выписка выражение высота выход выходной газ газета галстук генерал герой глава глаз глубина голос гонка гора город горячее господин гость государство граница гроза гром грудь группа грязь губа данные дверь движение двор девочка девушка дед действие деревня дерево десяток детство деятельность диагноз диван директор договор доктор документ должность долина доллар доля дорога доход дочь друг дядя жара жена женщина журнал завод задача зал зарплата защита заявление звезда звук здание здоровье зем